# Team Project (Title tbd)

#### `Ideas:`
- Does regular season win record impact playoff performance?
    - Is a winning record enough to do well in the postseason?
- Can a player actually "carry" a team?
    - 
- 

In [3]:
import pandas as pd
import plotly as px
import numpy as np
from sklearn.linear_model import LinearRegression
import plotly.express as pxo

In [18]:
per = pd.read_csv("per.csv", index_col="Player")
per = per.rename(columns={"PER▼":"PER"})
per

,Rk,Pos,Age,Tm,PER,3PAr,Player-additional
Player,,,,,,,
Nikola Jokić,1,C,27,DEN,31.5,0.146,jokicni01
Joel Embiid,2,C,28,PHI,31.4,0.151,embiijo01
Giannis Antetokounmpo,3,PF,28,MIL,29.0,0.134,antetgi01
Luka Dončić,4,PG,23,DAL,28.7,0.373,doncilu01
Anthony Davis,5,C,29,LAL,27.8,0.077,davisan02
...,...,...,...,...,...,...,...
Royce O'Neale,186,SF,29,BRK,9.2,0.707,onealro01
Gabe Vincent,187,PG,26,MIA,9.0,0.617,vincega01
Patrick Beverley,188,SG-PG,34,TOT,8.9,0.664,beverpa01


In [12]:
teams = pd.read_csv("teams.csv", index_col="Team")
teams

,Rk,Overall
Team,,
Milwaukee Bucks,1,58-24
Boston Celtics,2,57-25
Philadelphia 76ers,3,54-28
Denver Nuggets,4,53-29
Cleveland Cavaliers,5,51-31
Memphis Grizzlies,6,51-31
Sacramento Kings,7,48-34
New York Knicks,8,47-35
Brooklyn Nets,9,45-37


In [13]:
three_pt_atmpts = pd.read_csv("3pa.csv", index_col="Team")
three_pt_atmpts.rename(index=lambda s: s[:-1] if s.endswith("*") else s, inplace=True)
three_pt_atmpts

,Rk,G,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,FT,FTA,FT%,AST,TOV,PTS
Team,,,,,,,,,,,,,,,,,
Sacramento Kings,1.0,82,43.6,88.2,0.494,13.8,37.3,0.369,29.8,50.9,0.586,19.8,25.1,0.790,27.3,13.5,120.7
Golden State Warriors,2.0,82,43.1,90.2,0.479,16.6,43.2,0.385,26.5,47.0,0.564,16.0,20.2,0.794,29.8,16.3,118.9
Atlanta Hawks,3.0,82,44.6,92.4,0.483,10.8,30.5,0.352,33.9,61.8,0.548,18.5,22.6,0.818,25.0,12.9,118.4
Boston Celtics,4.0,82,42.2,88.8,0.475,16.0,42.6,0.377,26.2,46.2,0.567,17.5,21.6,0.812,26.7,13.4,117.9
Oklahoma City Thunder,5.0,82,43.1,92.6,0.465,12.1,34.1,0.356,31.0,58.5,0.530,19.2,23.7,0.809,24.4,13.0,117.5
Los Angeles Lakers,6.0,82,42.9,89.0,0.482,10.8,31.2,0.346,32.1,57.8,0.555,20.6,26.6,0.775,25.3,14.1,117.2
Utah Jazz,7.0,82,42.5,89.8,0.473,13.3,37.8,0.353,29.2,52.0,0.560,18.7,23.8,0.786,26.0,15.4,117.1
Memphis Grizzlies,8.0,82,43.7,92.1,0.475,12.0,34.2,0.351,31.7,57.9,0.548,17.5,23.8,0.733,26.0,13.6,116.9
Milwaukee Bucks,9.0,82,42.7,90.4,0.473,14.8,40.3,0.368,27.9,50.1,0.557,16.6,22.4,0.743,25.8,14.6,116.9


In [14]:
abbreviations = pd.read_csv("abbreviations.csv", index_col="name")

# Rename index and columns to preferred names
abbreviations.rename_axis(index={"name": "Team"}, inplace=True)
abbreviations.rename(columns={"prefix_1": "abbrev"}, inplace=True)

# Regularize abbreviations
abbreviations["abbrev"] = abbreviations["abbrev"].apply(lambda s: s.upper())

# Drop unneeded column
abbreviations.drop(columns="prefix_2", inplace=True)
abbreviations

,abbrev
Team,
Boston Celtics,BOS
Brooklyn Nets,BKN
New York Knicks,NY
Philadelphia 76ers,PHI
Toronto Raptors,TOR
Golden State Warriors,GS
Los Angeles Clippers,LAC
Los Angeles Lakers,LAL
Phoenix Suns,PHX


## Question 1: How does player efficiency rating (PER) correlate with a team's win-loss record?



In [26]:
model = LinearRegression()
display(per)
just_per = per["PER"]

teams[["wins", "losses"]] = teams["Overall"].str.split("-", expand=True).astype(int)
teams["ratio"] = round((teams["wins"] / teams["losses"]),2)
teams
# model.fit(just_per, )

,Rk,Pos,Age,Tm,PER,3PAr,Player-additional
Player,,,,,,,
Nikola Jokić,1,C,27,DEN,31.5,0.146,jokicni01
Joel Embiid,2,C,28,PHI,31.4,0.151,embiijo01
Giannis Antetokounmpo,3,PF,28,MIL,29.0,0.134,antetgi01
Luka Dončić,4,PG,23,DAL,28.7,0.373,doncilu01
Anthony Davis,5,C,29,LAL,27.8,0.077,davisan02
...,...,...,...,...,...,...,...
Royce O'Neale,186,SF,29,BRK,9.2,0.707,onealro01
Gabe Vincent,187,PG,26,MIA,9.0,0.617,vincega01
Patrick Beverley,188,SG-PG,34,TOT,8.9,0.664,beverpa01


,Rk,Overall,wins,losses,ratio
Team,,,,,
Milwaukee Bucks,1,58-24,58,24,2.42
Boston Celtics,2,57-25,57,25,2.28
Philadelphia 76ers,3,54-28,54,28,1.93
Denver Nuggets,4,53-29,53,29,1.83
Cleveland Cavaliers,5,51-31,51,31,1.65
Memphis Grizzlies,6,51-31,51,31,1.65
Sacramento Kings,7,48-34,48,34,1.41
New York Knicks,8,47-35,47,35,1.34
Brooklyn Nets,9,45-37,45,37,1.22


# Question 2: 